In [1]:
import os
import collections
import json
from PIL import Image
import PIL
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook as tq
import pickle
with open("test_rectangles.p","rb") as f:
        my_list = pickle.load(f)
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.models import Model
from keras.models import Sequential
from keras.layers import *
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras import callbacks
remote = callbacks.RemoteMonitor(root='http://localhost:9000')
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from keras.utils.visualize_util import plot
from keras.optimizers import *
%matplotlib inline
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
import csv
from sklearn.metrics import log_loss
from PIL import Image
import os
import random
import tarfile

Using Theano backend.
Using gpu device 0: GeForce GTX 960M (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5105)


In [2]:
# Open Image Net Images

def untar_imagenet():
    img_net_path = 'n02622955.tar'
    tar = tarfile.open(img_net_path)
    tar.extractall(path="./n02622955/")
    tar.close()

In [3]:
def erase_JPEG():
    for f in os.listdir('./'):
        if '.JPEG' in f:
            os.system('rm '+f)

In [4]:
# Try to create dataset from 
files = os.listdir('./')
lab_f_names = []
jsonFile = collections.defaultdict(list)
for f in files:
    if '.json' in f:
        lab_f_names.append(f)
for fjson in lab_f_names:
    with open(fjson,'r+') as f:
        jsonFile[fjson] = json.load(f)


In [9]:
binary_folder_train = './binary_dataset/train/'
binary_folder_test = './binary_dataset/test/'
train_path = './train/train/'
subFolder_train = [i for i in os.listdir(binary_folder_train) if i!= '.ipynb_checkpoints' and 'train' in i]
subFolder_test = [i for i in os.listdir(binary_folder_test) if i!= '.ipynb_checkpoints' and 'test' in i]
print(subFolder_train)
print(subFolder_test)

['fish_train', 'not_fish_train']
['fist_test', 'not_fish_test']


In [10]:
obj = [len(os.listdir(train_path+direc+'/')) for direc in os.listdir('./train/train/') if direc!='.DS_Store']
print(obj)
min_dir_images = obj[3]
print(min_dir_images)

[1719, 200, 117, 67, 465, 299, 176, 734]
67


In [11]:

def is_separate(rect_a, rect_b):
    try:
        xa,ya,xa2,ya2 = rect_a[0][0],rect_a[0][1],rect_a[1][0],rect_a[1][1]
        xb,yb,xb2,yb2 = rect_b[0][0],rect_b[0][1],rect_b[1][0],rect_b[1][1]   
        separate = xa2 < xb or ya2 < yb or xa > xb2 or ya > yb2
    except:
        print(rect_a, rect_b)
    if separate == True:
        return 0
    else:
        return 1

def transform_rect(rect,size):
    w,h = size[0],size[1]
    if int(rect[0]+rect[2]) <= w and int(rect[1]+rect[3]) <= h: 
        return [(rect[0],rect[1]),(rect[0]+rect[2],rect[1]+rect[3])]
    
def rect_generator(d_x,d_y,size):
    rects = []
    N_img = int(size[0]/d_x)
    for i in range(N_img):
        for j in range(N_img):
            rects.append(transform_rect([i*d_x,j*d_y,d_x,d_y],size))
    return rects
            
    
def cut_and_get_no_fish_img(d_x,d_y,poisson_pos,img):
    right_rects = []
    img_size = img.size
    rects = rect_generator(d_x,d_y,img_size) 
    if poisson_pos == []:
        return rects
    else:
        for recta in rects:
            if recta != None :
                good = 0
                for rectb in poisson_pos:
                    if transform_rect(rectb,img_size) != None:
                        good+=is_separate(recta,transform_rect(rectb,img_size))
                        if good == 1:
                            break
                if good == 0:
                    right_rects.append(recta)
        return right_rects
     
    

In [12]:
import matplotlib.pyplot as plt
N = 5
alpha = 0.8
for key in jsonFile:
    directory = key[:3].upper()+'/'
    print(directory)
    M = len(jsonFile[key])
    count = 0
    for element in tq(jsonFile[key]):
        if count < int(alpha*M): 
            filename = element['filename']
            if '/' in filename:
                filename = filename.split('/')[-1]
            annotations = element['annotations']
            img_path = train_path+directory+filename
            img = Image.open(img_path)
            size = img.size
            d_x,d_y = int(size[0]/N),int(size[1]/N)
            poisson_pos = []
            for annotation in annotations:
                x = annotation['x']
                y = annotation['y']
                w = annotation['width']
                h = annotation['height'] 
                poisson_pos.append([x,y,w,h])
            sub_rects = cut_and_get_no_fish_img(d_x,d_y,poisson_pos,img)
            # NO FISHES TRAIN 
            for k,rect in enumerate(sub_rects):
                x = rect[0][0]
                y = rect[0][1]
                x2 = rect[1][0]
                y2 = rect[1][1]
                img2 = img.crop((x, y, x2, y2))#.resize((size), PIL.Image.ANTIALIAS)
                output_name = binary_folder_train+subFolder_train[1]+'/'+str(k)+'_'+filename
                img2.save(output_name)
            # FISHES TRAIN 
            if poisson_pos != None and poisson_pos != []:
                try:
                    for k,rect in enumerate(poisson_pos):
                        rect_t = transform_rect(rect,size)
                        x = rect_t[0][0]
                        y = rect_t[0][1]
                        x2 = rect_t[1][0]
                        y2 = rect_t[1][1]
                        img2 = img.crop((x, y, x2, y2))#.resize((size), PIL.Image.ANTIALIAS)
                        output_name = binary_folder_train+subFolder_train[0]+'/'+str(k)+'_'+filename
                        img2.save(output_name)
                except:
                    continue
            count+=1
        else:
            filename = element['filename']
            if '/' in filename:
                filename = filename.split('/')[-1]
            annotations = element['annotations']
            img_path = train_path+directory+filename
            img = Image.open(img_path)
            size = img.size
            d_x,d_y = int(size[0]/N),int(size[1]/N)
            poisson_pos = []
            for annotation in annotations:
                x = annotation['x']
                y = annotation['y']
                w = annotation['width']
                h = annotation['height'] 
                poisson_pos.append([x,y,w,h])
            sub_rects = cut_and_get_no_fish_img(d_x,d_y,poisson_pos,img)
            # NO FISHES TRAIN 
            for k,rect in enumerate(sub_rects):
                x = rect[0][0]
                y = rect[0][1]
                x2 = rect[1][0]
                y2 = rect[1][1]
                img2 = img.crop((x, y, x2, y2))#.resize((size), PIL.Image.ANTIALIAS)
                output_name = binary_folder_test+subFolder_test[1]+'/'+str(k)+'_'+filename
                img2.save(output_name)
            # FISHES TRAIN 
            if poisson_pos != None and poisson_pos != []:
                try:
                    for k,rect in enumerate(poisson_pos):
                        rect_t = transform_rect(rect,size)
                        x = rect_t[0][0]
                        y = rect_t[0][1]
                        x2 = rect_t[1][0]
                        y2 = rect_t[1][1]
                        img2 = img.crop((x, y, x2, y2))#.resize((size), PIL.Image.ANTIALIAS)
                        output_name = binary_folder_test+subFolder_test[0]+'/'+str(k)+'_'+filename
                        img2.save(output_name)
                except:
                    continue
            count+=1
            

DOL/

ALB/

LAG/

BET/

YFT/



In [15]:
def untar_imagenet(name):
    img_net_path = name+'.tar'
    tar = tarfile.open(img_net_path)
    tar.extractall(path="./"+name+"/")
    tar.close()
untar_imagenet('n02512053')

In [16]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.models import Model
from keras.models import Sequential
from keras.layers import *
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras import callbacks
remote = callbacks.RemoteMonitor(root='http://localhost:9000')
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from keras.utils.visualize_util import plot
from keras.optimizers import *
%matplotlib inline
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
import csv
from sklearn.metrics import log_loss
from PIL import Image
import os
import random

In [36]:
#
binary_folder_train = './binary_dataset/train/'
binary_folder_train_fish = binary_folder_train+'fish'
binary_folder_train_not_fish = binary_folder_train+'not_fish'
binary_folder_val = './binary_dataset/test/'

data_gen_args = dict(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=True,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.,
    zoom_range=0.1,
    channel_shift_range=0.,
    fill_mode='nearest',
    cval=0.,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=None,
    dim_ordering=K.image_dim_ordering())

train_datagen = ImageDataGenerator(**data_gen_args)
validation_datagen = ImageDataGenerator(**data_gen_args)

train_generator = train_datagen.flow_from_directory(binary_folder_train, batch_size=16,save_to_dir=binary_folder_train_fish,target_size=(480,360),classes=['Fish'])
validation_generator = validation_datagen.flow_from_directory(binary_folder_val, batch_size=16,save_to_dir=binary_folder_val)

Found 3280 images belonging to 1 classes.
Found 11800 images belonging to 2 classes.


In [37]:
for X_batch, Y_batch in train_generator:
    if len(os.listdir(binary_folder_train_fish)) <= len(os.listdir(binary_folder_train_not_fish)):
        continue
    else:
        break

D:\Software\Anaconda3\lib\site-packages\keras\keras\preprocessing\image.py:346: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn'tbeen fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


In [44]:
binary_folder_train = './binary_dataset/train/'
binary_folder_train_fish = binary_folder_train+'fish'
binary_folder_train_not_fish = binary_folder_train+'not_fish'
binary_folder_test = './binary_dataset/test/'

data_gen_args = dict(
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.,
    zoom_range=0.1,
    )

train_datagen = ImageDataGenerator(**data_gen_args)
validation_datagen = ImageDataGenerator(**data_gen_args)
test_datagen = ImageDataGenerator(**data_gen_args)

train_generator = train_datagen.flow_from_directory(binary_folder_train, batch_size=16)
validation_generator = validation_datagen.flow_from_directory(binary_folder_train, batch_size=16)
test_generator = validation_datagen.flow_from_directory(binary_folder_test, batch_size=16)

Found 98943 images belonging to 2 classes.
Found 98943 images belonging to 2 classes.
Found 11800 images belonging to 2 classes.


In [48]:
model = Sequential()
model.add(Activation(activation="relu", input_shape=(3,None,None)))

model.add(Convolution2D(32, 5, 5, border_mode='same', activation='relu'))
model.add(ZeroPadding2D((1, 1))) 
model.add(Convolution2D(32, 5, 5, border_mode='same', activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
model.add(ZeroPadding2D((1, 1))) 
model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(Convolution2D(128, 3, 3, border_mode='same', activation='relu'))
model.add(ZeroPadding2D((1, 1))) 
model.add(Convolution2D(128, 3, 3, border_mode='same', activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(Convolution2D(256, 3, 3, border_mode='same', activation='relu'))
model.add(ZeroPadding2D((1, 1))) 
model.add(Convolution2D(256, 3, 3, border_mode='same', activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))


model.add(GlobalMaxPooling2D())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1))
model.add(Activation('softmax'))

In [49]:
# compile the model (should be done *after* setting layers to non-trainable)
opt = SGD(lr=1e-3, decay=1e-6, momentum=0.001, nesterov=True)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [50]:
# train the model on the new data for a few epochs
model.fit_generator(train_generator,
        samples_per_epoch=1024,
        nb_epoch=10,
        validation_data=validation_generator,
        verbose=0,
        nb_val_samples=256, callbacks=[remote])

Exception: Error when checking model target: expected activation_12 to have shape (None, 1) but got array with shape (16, 2)

In [ ]:
model.save('model2.h5')

In [ ]:
def get_result(result):
    max = 0
    j = 0 
    for i in range(len(result)):
        if max < result[i]:
            max = result[i]
            j = i
    index_max = j
    output = []
    for i in range(len(result)):
        if i == index_max:
            output.append(1)
        else:
            output.append(0) 
    return output
                   

In [ ]:
import time
result = []
t0 = time.time()
time_limit = 10
for X_batch, Y_batch in test_generator:
    results = model.predict(X_batch)
    j = 0
    for i in range(len(results)):
        im = np.rollaxis(np.array(X_batch[i]), 0, 3) *-1
        np.roll(im, 1, axis=-1)
        plt.imshow(im,cmap=plt.cm.gray)
        plt.show()
        print("True value:" + str(Y_batch[i]))
        print("Predicted value" + str(get_result(results[i])))
        if str(get_result(results[i])) == str([int(n) for n in list(Y_batch[i])]):
            j+=1
        if i == len(results) -1:
            result.append(j/len(results))
    t1 = time.time()
    if (t1-t0) > time_limit:
        break
sum=0
min = 1 
max = 0
print(result)
for n in result:
    if n < min:
        min = n
    if n > max:
        max = n
    sum+=n
print(len(result),sum/len(result),max,min)